# Earthquake Ruptures

##  Configuration

In [ ]:
# Import dependencies
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from get_rupture_info import repture_xml_to_df

## User input

In [ ]:
# Event name
event = '20150425_M7.8_Gorkha'

## Read OQ rupture data

In [ ]:
# Select all rupture files in OQ format
in_path = os.path.join(event, 'Rupture')
rup_files = glob.glob(os.path.join('..', '*', '*', in_path, 'earthquake_rupture_model*.xml'))

folder = os.path.dirname(rup_files[0])

df = [ ]
rups = {}
for rupfile in sorted(rup_files):
    source = os.path.basename(rupfile)[25:-4]
    # print(source)
    rups[source], rup_df = repture_xml_to_df(rupfile)
    rup_df.index = [source]
    df.append(rup_df)

df = pd.concat(df)
df.index.name = 'source'

# Round coordinates estimated by OQ
cols = ['strike', 'dip', 'rake']
df[cols] = df[cols].round(2)

df

### Rupture mechanism and trt
Read rupture details from `earthquake_information.csv`

In [ ]:
eq_info = glob.glob(os.path.join('..', '*', event, 'earthquake_information.csv'))
info = pd.read_csv(eq_info[0], index_col='FIELD')
mechanism = info.loc['Fault_mechanism', 'DESCRIPTION']
trt = info.loc['Tectonic_region_type', 'DESCRIPTION']

print(f'Fault_mechanism: {mechanism}') 
print(f'Tectonic_region_type: {trt}')

## Plot ruptures

In [ ]:
# Plot rupture geometry and hypocenter
fig = plt.figure(figsize=(10,10))

for source, rup in rups.items():
    try:
        plt.plot(*rup.surface.get_surface_boundaries(), label=source)
    except AttributeError as error:
        print(f'No possible to plot {source} geometry:\n', error)

plt.gca().set_prop_cycle(None)

for source, rup in rups.items():
    plt.plot(rup.hypocenter.longitude, rup.hypocenter.latitude,
             '*', markersize=10)

plt.legend(loc='best')
plt.axis('equal')

# Save figure
fig.savefig(os.path.join(folder, 'earthquake_ruptures.png'),
            dpi=300,
            bbox_inches='tight')

## Update README

In [ ]:
# Visualise README file
readme = os.path.join(folder, 'README.md')

# Read original README
f = open(readme, 'r')
content = f.read()
f.close
print(*content.split('\n'), sep='\n')

### Update `Rupture information` table

In [ ]:
# Update README
f=open(os.path.join(folder, 'README.md'), 'w')

# md table with rupture details
details = pd.DataFrame({'atribute': ['Fault mechanism', 'Tectonic region type'], 
                        'description': [mechanism, trt]})
rup_details = details.to_markdown(index=False)

# md table with rupture nodal plane
cols = ['longitude', 'latitude', 'depth', 'strike', 'dip', 'rake', 'mag']
rup_hyps = df[cols].fillna('').to_markdown()

# Update README file
to_add = f'''# RUPTURE INFORMATION
    
## Rupture details\n
{rup_details}

### Preferred nodal plane solution\n
{rup_hyps}

### Rupture figure\n
![](earthquake_ruptures.png)

'''
f.write(to_add)

# Include references
refs = content[content.find('## References\n'):]
f.write(refs)
f.close()

# Print new README
f = open(readme, 'r')
content = f.read()
f.close
print('\n', *content.split('\n'), sep='\n')